In [ ]:
#clear all
%reset -f

#import packages
import numpy as np
import scipy
import sys
import os
import pandas as pd
import mne
import matplotlib
import h5py
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'frn'
roi = 'medfro'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [ ]:
#setting up path/ directory
#access trial type epoched data for each participant
def load_tfr_epochs(pp_num, root_dir, erp_path, task):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'p%03d_%s-epo.fif' % (pp_num, task))

    epochs = mne.read_epochs(filename)

    return epochs, pp_directory

In [ ]:
#set up parameters for time-frequency morlet convolution
def set_tfr_params(l_freq = 6, h_freq = 35, num_freq = 50, cycle = 6, samp_rate = 200):
    freqs = np.logspace(*np.log10([l_freq, h_freq]), num = num_freq)
    n_cycles = cycle
    sfreq = samp_rate
    
    return freqs, n_cycles, sfreq

In [ ]:
#save tfr'd data
def save_tfr_data(pp_num, root_dir, erp_path, data, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    out_fname = os.path.join(pp_directory, 'p%03d_%s_%s-tfr.h5' % (pp_num, task, output))
    mne.time_frequency.write_tfrs(out_fname, tfr=data, overwrite = True)

In [ ]:
#load tfr'd data
def load_tfr_data(pp_num, root_dir, erp_path, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    out_fname = os.path.join(pp_directory, 'p%03d_%s_%s-tfr.h5' % (pp_num, task, output))
    dat = mne.time_frequency.read_tfrs(out_fname)
    
    return dat

In [ ]:
# After transforming the TFR data into a single signal representative of frequency and channels
# we compare the two signals: same as taking their difference (signal1 minus signal2) and using this in permutation test
def get_clust_perm_test(conditionA, conditionB, pval, n_permutations):
    #take difference of two conditions
    data = np.subtract(conditionA, conditionB)
    np.shape(data)
    #define cluster forming threshold based on p-value
    df = len(participants) - 1
    thresh = scipy.stats.t.ppf(1 - pval / 2, df)
    #run cluster-based permutation test
    T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh, 
                                                          n_permutations = n_permutations, tail = 0, 
                                                          adjacency = None, seed = 999, 
                                                          out_type = 'mask', verbose = True)

    return T_0, clust_idx, clust_pvals, H0

In [ ]:
# # save average tfr per participant for each condition - run if not yet implemented
# # morlet wavelet convolution for each participant, according to trial types - POWER

# perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

# for pcond in range(0, len(perturb_conds)):
#     for pp in participants:
#         data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
# #         #use all trials for aligned
# #         if pcond == 1:
# #             data = data[0:12] #first 12/45 trials for early
# #         elif pcond == 2:
# #             data = data[33:45] #last 12/45 trials for late
# #         elif pcond == 3:
# #             data = data[0:12]
# #         elif pcond == 4:
# #             data = data[33:45]
# #         elif pcond == 5:
# #             data = data[0:12]
# #         elif pcond == 6:
# #             data = data[36:48] #rdm has 48 trials only
        
#         freqs, n_cycles, sfreq = set_tfr_params()
        
#         #average epochs within participant
#         power = mne.time_frequency.tfr_morlet(
#             data,
#             freqs=freqs,
#             n_cycles=n_cycles,
#             use_fft=False,
#             decim=1,
#             n_jobs=None,
#             return_itc = False,
#             average = True
#         )
        
#         save_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, data = power, task = perturb_conds[pcond], output = "power")

In [ ]:
# load in tfr object, ensure to baseline correct; (tfr shape is channels, freqs, timepts)
# narrow down to channels 
# narrow down to frequencies and timepts
# take mean of theta, alpha, or beta ranges (freqs)
# take mean across electrodes (channels)

perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

# medial frontal CHANNELS
channels = ['F1', 'Fz', 'F2',
            'FC1', 'FCz', 'FC2',
            'C1', 'Cz', 'C2']


# BASELINE FOR FEEDBACK ONSET
baseline_t = (-0.1, 0)

# THETA
freq_lower = 6
freq_upper = 8

theta_medfro_EarlyLateAligned = []
theta_medfro_EarlyRot = []
theta_medfro_LateRot = []
theta_medfro_EarlyMir = []
theta_medfro_LateMir = []
theta_medfro_EarlyRdm = []
theta_medfro_LateRdm = []
full_theta_medfro_EarlyLateAligned = []
full_theta_medfro_EarlyRot = []
full_theta_medfro_LateRot = []
full_theta_medfro_EarlyMir = []
full_theta_medfro_LateMir = []
full_theta_medfro_EarlyRdm = []
full_theta_medfro_LateRdm = []

for pcond in range(0, len(perturb_conds)):
    medfroppdat = []
    full_medfroppdat = []
    for pp in participants:
        dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond], output = "power")
        dat = dat[0]*1e12 #convert V^2 to uV^2
        # apply baseline correction
        dat = dat.apply_baseline(baseline_t, mode='mean')
        # narrow down to channels
        dat = dat.pick_channels(channels)
        #transform to ndarray (channels, freqs, times)
        npdat = dat.data

        # get needed frequency indices
        nfreqs = [] #get indices of frequencies we want
        for i in range(0, len(dat.freqs)):
            if dat.freqs[i] >= freq_lower and dat.freqs[i] <= freq_upper:
                nfreqs.append(i)
        npdat = npdat[:, nfreqs, :]
        full_npdat = npdat #all timepts included

        # get needed timept indices
        ntimes = list(range(400,601))
        npdat = npdat[:,:,ntimes]
        
        ppdat = np.mean(npdat, axis = 1) #calculate mean across frequencies
        ppdat = np.mean(ppdat, axis = 0) # calculate mean across channels
        
        full_ppdat = np.mean(full_npdat, axis = 1)
        full_ppdat = np.mean(full_ppdat, axis = 0)
        
        medfroppdat.append(ppdat)
        full_medfroppdat.append(full_ppdat)
        
    if pcond == 0:
        theta_medfro_EarlyLateAligned.append(medfroppdat)
        full_theta_medfro_EarlyLateAligned.append(full_medfroppdat)
    elif pcond == 1:
        theta_medfro_EarlyRot.append(medfroppdat)
        full_theta_medfro_EarlyRot.append(full_medfroppdat)
    elif pcond == 2:
        theta_medfro_LateRot.append(medfroppdat)
        full_theta_medfro_LateRot.append(full_medfroppdat)
    elif pcond == 3:
        theta_medfro_EarlyMir.append(medfroppdat)
        full_theta_medfro_EarlyMir.append(full_medfroppdat)
    elif pcond == 4:
        theta_medfro_LateMir.append(medfroppdat)
        full_theta_medfro_LateMir.append(full_medfroppdat)
    elif pcond == 5:
        theta_medfro_EarlyRdm.append(medfroppdat)
        full_theta_medfro_EarlyRdm.append(full_medfroppdat)
    elif pcond == 6:
        theta_medfro_LateRdm.append(medfroppdat)
        full_theta_medfro_LateRdm.append(full_medfroppdat)

# ALPHA
freq_lower = 9
freq_upper = 13

alpha_medfro_EarlyLateAligned = []
alpha_medfro_EarlyRot = []
alpha_medfro_LateRot = []
alpha_medfro_EarlyMir = []
alpha_medfro_LateMir = []
alpha_medfro_EarlyRdm = []
alpha_medfro_LateRdm = []
full_alpha_medfro_EarlyLateAligned = []
full_alpha_medfro_EarlyRot = []
full_alpha_medfro_LateRot = []
full_alpha_medfro_EarlyMir = []
full_alpha_medfro_LateMir = []
full_alpha_medfro_EarlyRdm = []
full_alpha_medfro_LateRdm = []

for pcond in range(0, len(perturb_conds)):
    medfroppdat = []
    full_medfroppdat = []
    for pp in participants:
        dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond], output = "power")
        dat = dat[0]*1e12 #convert V^2 to uV^2
        # apply baseline correction
        dat = dat.apply_baseline(baseline_t, mode='mean')
        # narrow down to channels
        dat = dat.pick_channels(channels)
        #transform to ndarray (channels, freqs, times)
        npdat = dat.data

        # get needed frequency indices
        nfreqs = [] #get indices of frequencies we want
        for i in range(0, len(dat.freqs)):
            if dat.freqs[i] >= freq_lower and dat.freqs[i] <= freq_upper:
                nfreqs.append(i)
        npdat = npdat[:, nfreqs, :]
        full_npdat = npdat #all timepts included

        # get needed timept indices
        ntimes = list(range(400,601))
        npdat = npdat[:,:,ntimes]
        
        ppdat = np.mean(npdat, axis = 1) #calculate mean across frequencies
        ppdat = np.mean(ppdat, axis = 0) # calculate mean across channels
        
        full_ppdat = np.mean(full_npdat, axis = 1)
        full_ppdat = np.mean(full_ppdat, axis = 0)
        
        medfroppdat.append(ppdat)
        full_medfroppdat.append(full_ppdat)
        
    if pcond == 0:
        alpha_medfro_EarlyLateAligned.append(medfroppdat)
        full_alpha_medfro_EarlyLateAligned.append(full_medfroppdat)
    elif pcond == 1:
        alpha_medfro_EarlyRot.append(medfroppdat)
        full_alpha_medfro_EarlyRot.append(full_medfroppdat)
    elif pcond == 2:
        alpha_medfro_LateRot.append(medfroppdat)
        full_alpha_medfro_LateRot.append(full_medfroppdat)
    elif pcond == 3:
        alpha_medfro_EarlyMir.append(medfroppdat)
        full_alpha_medfro_EarlyMir.append(full_medfroppdat)
    elif pcond == 4:
        alpha_medfro_LateMir.append(medfroppdat)
        full_alpha_medfro_LateMir.append(full_medfroppdat)
    elif pcond == 5:
        alpha_medfro_EarlyRdm.append(medfroppdat)
        full_alpha_medfro_EarlyRdm.append(full_medfroppdat)
    elif pcond == 6:
        alpha_medfro_LateRdm.append(medfroppdat)
        full_alpha_medfro_LateRdm.append(full_medfroppdat)

# BETA
freq_lower = 13
freq_upper = 25

beta_medfro_EarlyLateAligned = []
beta_medfro_EarlyRot = []
beta_medfro_LateRot = []
beta_medfro_EarlyMir = []
beta_medfro_LateMir = []
beta_medfro_EarlyRdm = []
beta_medfro_LateRdm = []
full_beta_medfro_EarlyLateAligned = []
full_beta_medfro_EarlyRot = []
full_beta_medfro_LateRot = []
full_beta_medfro_EarlyMir = []
full_beta_medfro_LateMir = []
full_beta_medfro_EarlyRdm = []
full_beta_medfro_LateRdm = []

for pcond in range(0, len(perturb_conds)):
    medfroppdat = []
    full_medfroppdat = []
    for pp in participants:
        dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond], output = "power")
        dat = dat[0]*1e12 #convert V^2 to uV^2
        # apply baseline correction
        dat = dat.apply_baseline(baseline_t, mode='mean')
        # narrow down to channels
        dat = dat.pick_channels(channels)
        #transform to ndarray (channels, freqs, times)
        npdat = dat.data

        # get needed frequency indices
        nfreqs = [] #get indices of frequencies we want
        for i in range(0, len(dat.freqs)):
            if dat.freqs[i] >= freq_lower and dat.freqs[i] <= freq_upper:
                nfreqs.append(i)
        npdat = npdat[:, nfreqs, :]
        full_npdat = npdat #all timepts included

        # get needed timept indices
        ntimes = list(range(400,601))
        npdat = npdat[:,:,ntimes]
        
        ppdat = np.mean(npdat, axis = 1) #calculate mean across frequencies
        ppdat = np.mean(ppdat, axis = 0) # calculate mean across channels
        
        full_ppdat = np.mean(full_npdat, axis = 1)
        full_ppdat = np.mean(full_ppdat, axis = 0)
        
        medfroppdat.append(ppdat)
        full_medfroppdat.append(full_ppdat)
        
    if pcond == 0:
        beta_medfro_EarlyLateAligned.append(medfroppdat)
        full_beta_medfro_EarlyLateAligned.append(full_medfroppdat)
    elif pcond == 1:
        beta_medfro_EarlyRot.append(medfroppdat)
        full_beta_medfro_EarlyRot.append(full_medfroppdat)
    elif pcond == 2:
        beta_medfro_LateRot.append(medfroppdat)
        full_beta_medfro_LateRot.append(full_medfroppdat)
    elif pcond == 3:
        beta_medfro_EarlyMir.append(medfroppdat)
        full_beta_medfro_EarlyMir.append(full_medfroppdat)
    elif pcond == 4:
        beta_medfro_LateMir.append(medfroppdat)
        full_beta_medfro_LateMir.append(full_medfroppdat)
    elif pcond == 5:
        beta_medfro_EarlyRdm.append(medfroppdat)
        full_beta_medfro_EarlyRdm.append(full_medfroppdat)
    elif pcond == 6:
        beta_medfro_LateRdm.append(medfroppdat)
        full_beta_medfro_LateRdm.append(full_medfroppdat)
    

In [ ]:
# Get timepts to use for indices given by cluster-based permutation later and for timepts in saved data frames
# But only grab 0 to 1 sec time-locked to feedback onset (idx= 400:601)
root_directory = root
pp = 0 #only need one participant

# we can use aligned data
dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = 'early_late_aligned', output = "power")
full_time = dat[0].times
time = full_time[400:601] #get only timepoints we want

In [ ]:
# save tfr data across participants in each condition (used for plotting signals)
theta_flists = [full_theta_medfro_EarlyLateAligned, full_theta_medfro_EarlyRot, full_theta_medfro_LateRot, 
               full_theta_medfro_EarlyRdm, full_theta_medfro_LateRdm,
               full_theta_medfro_EarlyMir, full_theta_medfro_LateMir]
alpha_flists = [full_alpha_medfro_EarlyLateAligned, full_alpha_medfro_EarlyRot, full_alpha_medfro_LateRot, 
                full_alpha_medfro_EarlyRdm, full_alpha_medfro_LateRdm, 
                full_alpha_medfro_EarlyMir, full_alpha_medfro_LateMir]
beta_flists = [full_beta_medfro_EarlyLateAligned, full_beta_medfro_EarlyRot, full_beta_medfro_LateRot, 
               full_beta_medfro_EarlyRdm, full_beta_medfro_LateRdm,
               full_beta_medfro_EarlyMir, full_beta_medfro_LateMir]

theta_conditionnames = ['theta_earlylate_aligned', 'theta_earlyrot', 'theta_laterot', 'theta_earlyrdm', 'theta_laterdm', 'theta_earlymir', 'theta_latemir']
alpha_conditionnames = ['alpha_earlylate_aligned', 'alpha_earlyrot', 'alpha_laterot', 'alpha_earlyrdm', 'alpha_laterdm', 'alpha_earlymir', 'alpha_latemir']
beta_conditionnames = ['beta_earlylate_aligned', 'beta_earlyrot', 'beta_laterot', 'beta_earlyrdm', 'beta_laterdm', 'beta_earlymir', 'beta_latemir']

freq_bands = ['theta', 'alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(theta_flists)):
            df = pd.DataFrame(theta_flists[f][0]).T
            df['time'] = full_time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_%s_%s.csv' % (theta_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 1:
        for f in range(0, len(alpha_flists)):
            df = pd.DataFrame(alpha_flists[f][0]).T
            df['time'] = full_time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_%s_%s.csv' % (alpha_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 2:
        for f in range(0, len(beta_flists)):
            df = pd.DataFrame(beta_flists[f][0]).T
            df['time'] = full_time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_%s_%s.csv' % (beta_conditionnames[f], erps))
            df.to_csv(filename)
            

In [ ]:
# First, we can compare each condition to aligned
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in plots

theta_flists = [theta_medfro_EarlyRot, theta_medfro_LateRot, 
               theta_medfro_EarlyRdm, theta_medfro_LateRdm,
               theta_medfro_EarlyMir, theta_medfro_LateMir]
alpha_flists = [alpha_medfro_EarlyRot, alpha_medfro_LateRot, 
                alpha_medfro_EarlyRdm, alpha_medfro_LateRdm, 
                alpha_medfro_EarlyMir, alpha_medfro_LateMir]
beta_flists = [beta_medfro_EarlyRot, beta_medfro_LateRot, 
               beta_medfro_EarlyRdm, beta_medfro_LateRdm,
               beta_medfro_EarlyMir, beta_medfro_LateMir]

theta_conditionnames = ['theta_earlyrot', 'theta_laterot', 'theta_earlyrdm', 'theta_laterdm', 'theta_earlymir', 'theta_latemir']
alpha_conditionnames = ['alpha_earlyrot', 'alpha_laterot', 'alpha_earlyrdm', 'alpha_laterdm', 'alpha_earlymir', 'alpha_latemir']
beta_conditionnames = ['beta_earlyrot', 'beta_laterot', 'beta_earlyrdm', 'beta_laterdm', 'beta_earlymir', 'beta_latemir']

freq_bands = ['theta', 'alpha', 'beta']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(theta_flists)):
            T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(theta_flists[f][0], theta_medfro_EarlyLateAligned[0], p, perms)
#             print(clust_idx, clust_pvals)
            if len(clust_idx) == 0:
                condition.append(theta_conditionnames[f])
                clust_idx_start.append(np.nan)
                clust_idx_end.append(np.nan)
                time_start.append(np.nan)
                time_end.append(np.nan)
                p_values.append(np.nan)
            else:
                for clust in range(0, len(clust_idx)):
                    cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                    cluster_start = cluster.start
                    clust_idx_start.append(cluster_start)
        
                    cluster_end = cluster.stop
                    clust_idx_end.append(cluster_end)
        
                    time_idx_start = time[cluster_start]
                    time_start.append(time_idx_start)
        
                    time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                    time_end.append(time_idx_end)
        
                    clust_p = clust_pvals[clust]
                    p_values.append(clust_p)
        
                    condition.append(theta_conditionnames[f])
            
    elif band == 1:
        for f in range(0, len(alpha_flists)):
            T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_flists[f][0], alpha_medfro_EarlyLateAligned[0], p, perms)
#             print(clust_idx, clust_pvals)
            if len(clust_idx) == 0:
                condition.append(alpha_conditionnames[f])
                clust_idx_start.append(np.nan)
                clust_idx_end.append(np.nan)
                time_start.append(np.nan)
                time_end.append(np.nan)
                p_values.append(np.nan)
            else:
                for clust in range(0, len(clust_idx)):
                    cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                    cluster_start = cluster.start
                    clust_idx_start.append(cluster_start)
        
                    cluster_end = cluster.stop
                    clust_idx_end.append(cluster_end)
        
                    time_idx_start = time[cluster_start]
                    time_start.append(time_idx_start)
        
                    time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                    time_end.append(time_idx_end)
        
                    clust_p = clust_pvals[clust]
                    p_values.append(clust_p)
        
                    condition.append(alpha_conditionnames[f])
    elif band == 2:
        for f in range(0, len(beta_flists)):
            T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_flists[f][0], beta_medfro_EarlyLateAligned[0], p, perms)
#             print(clust_idx, clust_pvals)
            if len(clust_idx) == 0:
                condition.append(beta_conditionnames[f])
                clust_idx_start.append(np.nan)
                clust_idx_end.append(np.nan)
                time_start.append(np.nan)
                time_end.append(np.nan)
                p_values.append(np.nan)
            else:
                for clust in range(0, len(clust_idx)):
                    cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                    cluster_start = cluster.start
                    clust_idx_start.append(cluster_start)
        
                    cluster_end = cluster.stop
                    clust_idx_end.append(cluster_end)
        
                    time_idx_start = time[cluster_start]
                    time_start.append(time_idx_start)
        
                    time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                    time_end.append(time_idx_end)
        
                    clust_p = clust_pvals[clust]
                    p_values.append(clust_p)
        
                    condition.append(beta_conditionnames[f])
        
perm_test = pd.DataFrame(
    {'condition': condition,
        'clust_idx_start': clust_idx_start,
        'clust_idx_end': clust_idx_end,
        'time_start': time_start,
        'time_end': time_end,
        'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_Permutation_test_vsAligned_%s_%s.csv' % (erps, roi))
perm_test.to_csv(perm_test_filename)


In [ ]:
# Next, we subtract aligned from each condition, so that we can compare early vs late in each perturbation type

freq_bands = ['theta', 'alpha', 'beta']
diffconds = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']

theta_medfro_earlyrot_diff = []
theta_medfro_laterot_diff = []
theta_medfro_earlyrdm_diff = []
theta_medfro_laterdm_diff = []
theta_medfro_earlymir_diff = []
theta_medfro_latemir_diff = []
full_theta_medfro_earlyrot_diff = []
full_theta_medfro_laterot_diff = []
full_theta_medfro_earlyrdm_diff = []
full_theta_medfro_laterdm_diff = []
full_theta_medfro_earlymir_diff = []
full_theta_medfro_latemir_diff = []

alpha_medfro_earlyrot_diff = []
alpha_medfro_laterot_diff = []
alpha_medfro_earlyrdm_diff = []
alpha_medfro_laterdm_diff = []
alpha_medfro_earlymir_diff = []
alpha_medfro_latemir_diff = []
full_alpha_medfro_earlyrot_diff = []
full_alpha_medfro_laterot_diff = []
full_alpha_medfro_earlyrdm_diff = []
full_alpha_medfro_laterdm_diff = []
full_alpha_medfro_earlymir_diff = []
full_alpha_medfro_latemir_diff = []

beta_medfro_earlyrot_diff = []
beta_medfro_laterot_diff = []
beta_medfro_earlyrdm_diff = []
beta_medfro_laterdm_diff = []
beta_medfro_earlymir_diff = []
beta_medfro_latemir_diff = []
full_beta_medfro_earlyrot_diff = []
full_beta_medfro_laterot_diff = []
full_beta_medfro_earlyrdm_diff = []
full_beta_medfro_laterdm_diff = []
full_beta_medfro_earlymir_diff = []
full_beta_medfro_latemir_diff = []

for band in range(0, len(freq_bands)):
    if band == 0:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(theta_medfro_EarlyRot[0], theta_medfro_EarlyLateAligned[0])
                theta_medfro_earlyrot_diff.append(diffevks)
                theta_medfro_earlyrot_diff = theta_medfro_earlyrot_diff[0] #to keep shape of object consistent
                
                full_diffevks = np.subtract(full_theta_medfro_EarlyRot[0], full_theta_medfro_EarlyLateAligned[0])
                full_theta_medfro_earlyrot_diff.append(full_diffevks)
                full_theta_medfro_earlyrot_diff = full_theta_medfro_earlyrot_diff[0] #to keep shape of object consistent
                
            elif cond == 1:
                diffevks = np.subtract(theta_medfro_LateRot[0], theta_medfro_EarlyLateAligned[0])
                theta_medfro_laterot_diff.append(diffevks)
                theta_medfro_laterot_diff = theta_medfro_laterot_diff[0]
                
                full_diffevks = np.subtract(full_theta_medfro_LateRot[0], full_theta_medfro_EarlyLateAligned[0])
                full_theta_medfro_laterot_diff.append(full_diffevks)
                full_theta_medfro_laterot_diff = full_theta_medfro_laterot_diff[0]
                
            elif cond == 2:
                diffevks = np.subtract(theta_medfro_EarlyRdm[0], theta_medfro_EarlyLateAligned[0])
                theta_medfro_earlyrdm_diff.append(diffevks)
                theta_medfro_earlyrdm_diff = theta_medfro_earlyrdm_diff[0]
                
                full_diffevks = np.subtract(full_theta_medfro_EarlyRdm[0], full_theta_medfro_EarlyLateAligned[0])
                full_theta_medfro_earlyrdm_diff.append(full_diffevks)
                full_theta_medfro_earlyrdm_diff = full_theta_medfro_earlyrdm_diff[0]
                
            elif cond == 3:
                diffevks = np.subtract(theta_medfro_LateRdm[0], theta_medfro_EarlyLateAligned[0])
                theta_medfro_laterdm_diff.append(diffevks)
                theta_medfro_laterdm_diff = theta_medfro_laterdm_diff[0]
                
                full_diffevks = np.subtract(full_theta_medfro_LateRdm[0], full_theta_medfro_EarlyLateAligned[0])
                full_theta_medfro_laterdm_diff.append(full_diffevks)
                full_theta_medfro_laterdm_diff = full_theta_medfro_laterdm_diff[0]
                
            elif cond == 4:
                diffevks = np.subtract(theta_medfro_EarlyMir[0], theta_medfro_EarlyLateAligned[0])
                theta_medfro_earlymir_diff.append(diffevks)
                theta_medfro_earlymir_diff = theta_medfro_earlymir_diff[0]
                
                full_diffevks = np.subtract(full_theta_medfro_EarlyMir[0], full_theta_medfro_EarlyLateAligned[0])
                full_theta_medfro_earlymir_diff.append(full_diffevks)
                full_theta_medfro_earlymir_diff = full_theta_medfro_earlymir_diff[0]
                
            elif cond == 5:
                diffevks = np.subtract(theta_medfro_LateMir[0], theta_medfro_EarlyLateAligned[0])
                theta_medfro_latemir_diff.append(diffevks)
                theta_medfro_latemir_diff = theta_medfro_latemir_diff[0]
                
                full_diffevks = np.subtract(full_theta_medfro_LateMir[0], full_theta_medfro_EarlyLateAligned[0])
                full_theta_medfro_latemir_diff.append(full_diffevks)
                full_theta_medfro_latemir_diff = full_theta_medfro_latemir_diff[0]
                
    elif band == 1:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(alpha_medfro_EarlyRot[0], alpha_medfro_EarlyLateAligned[0])
                alpha_medfro_earlyrot_diff.append(diffevks)
                alpha_medfro_earlyrot_diff = alpha_medfro_earlyrot_diff[0] #to keep shape of object consistent
                
                full_diffevks = np.subtract(full_alpha_medfro_EarlyRot[0], full_alpha_medfro_EarlyLateAligned[0])
                full_alpha_medfro_earlyrot_diff.append(full_diffevks)
                full_alpha_medfro_earlyrot_diff = full_alpha_medfro_earlyrot_diff[0] #to keep shape of object consistent
                
            elif cond == 1:
                diffevks = np.subtract(alpha_medfro_LateRot[0], alpha_medfro_EarlyLateAligned[0])
                alpha_medfro_laterot_diff.append(diffevks)
                alpha_medfro_laterot_diff = alpha_medfro_laterot_diff[0]
                
                full_diffevks = np.subtract(full_alpha_medfro_LateRot[0], full_alpha_medfro_EarlyLateAligned[0])
                full_alpha_medfro_laterot_diff.append(full_diffevks)
                full_alpha_medfro_laterot_diff = full_alpha_medfro_laterot_diff[0]
                
            elif cond == 2:
                diffevks = np.subtract(alpha_medfro_EarlyRdm[0], alpha_medfro_EarlyLateAligned[0])
                alpha_medfro_earlyrdm_diff.append(diffevks)
                alpha_medfro_earlyrdm_diff = alpha_medfro_earlyrdm_diff[0]
                
                full_diffevks = np.subtract(full_alpha_medfro_EarlyRdm[0], full_alpha_medfro_EarlyLateAligned[0])
                full_alpha_medfro_earlyrdm_diff.append(full_diffevks)
                full_alpha_medfro_earlyrdm_diff = full_alpha_medfro_earlyrdm_diff[0]
                
            elif cond == 3:
                diffevks = np.subtract(alpha_medfro_LateRdm[0], alpha_medfro_EarlyLateAligned[0])
                alpha_medfro_laterdm_diff.append(diffevks)
                alpha_medfro_laterdm_diff = alpha_medfro_laterdm_diff[0]
                
                full_diffevks = np.subtract(full_alpha_medfro_LateRdm[0], full_alpha_medfro_EarlyLateAligned[0])
                full_alpha_medfro_laterdm_diff.append(full_diffevks)
                full_alpha_medfro_laterdm_diff = full_alpha_medfro_laterdm_diff[0]
                
            elif cond == 4:
                diffevks = np.subtract(alpha_medfro_EarlyMir[0], alpha_medfro_EarlyLateAligned[0])
                alpha_medfro_earlymir_diff.append(diffevks)
                alpha_medfro_earlymir_diff = alpha_medfro_earlymir_diff[0]
                
                full_diffevks = np.subtract(full_alpha_medfro_EarlyMir[0], full_alpha_medfro_EarlyLateAligned[0])
                full_alpha_medfro_earlymir_diff.append(full_diffevks)
                full_alpha_medfro_earlymir_diff = full_alpha_medfro_earlymir_diff[0]
                
            elif cond == 5:
                diffevks = np.subtract(alpha_medfro_LateMir[0], alpha_medfro_EarlyLateAligned[0])
                alpha_medfro_latemir_diff.append(diffevks)
                alpha_medfro_latemir_diff = alpha_medfro_latemir_diff[0]
                
                full_diffevks = np.subtract(full_alpha_medfro_LateMir[0], full_alpha_medfro_EarlyLateAligned[0])
                full_alpha_medfro_latemir_diff.append(full_diffevks)
                full_alpha_medfro_latemir_diff = full_alpha_medfro_latemir_diff[0]
                
    elif band == 2:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(beta_medfro_EarlyRot[0], beta_medfro_EarlyLateAligned[0])
                beta_medfro_earlyrot_diff.append(diffevks)
                beta_medfro_earlyrot_diff = beta_medfro_earlyrot_diff[0] #to keep shape of object consistent
                
                full_diffevks = np.subtract(full_beta_medfro_EarlyRot[0], full_beta_medfro_EarlyLateAligned[0])
                full_beta_medfro_earlyrot_diff.append(full_diffevks)
                full_beta_medfro_earlyrot_diff = full_beta_medfro_earlyrot_diff[0] #to keep shape of object consistent
                
            elif cond == 1:
                diffevks = np.subtract(beta_medfro_LateRot[0], beta_medfro_EarlyLateAligned[0])
                beta_medfro_laterot_diff.append(diffevks)
                beta_medfro_laterot_diff = beta_medfro_laterot_diff[0]
                
                full_diffevks = np.subtract(full_beta_medfro_LateRot[0], full_beta_medfro_EarlyLateAligned[0])
                full_beta_medfro_laterot_diff.append(full_diffevks)
                full_beta_medfro_laterot_diff = full_beta_medfro_laterot_diff[0]
                
            elif cond == 2:
                diffevks = np.subtract(beta_medfro_EarlyRdm[0], beta_medfro_EarlyLateAligned[0])
                beta_medfro_earlyrdm_diff.append(diffevks)
                beta_medfro_earlyrdm_diff = beta_medfro_earlyrdm_diff[0]
                
                full_diffevks = np.subtract(full_beta_medfro_EarlyRdm[0], full_beta_medfro_EarlyLateAligned[0])
                full_beta_medfro_earlyrdm_diff.append(full_diffevks)
                full_beta_medfro_earlyrdm_diff = full_beta_medfro_earlyrdm_diff[0]
                
            elif cond == 3:
                diffevks = np.subtract(beta_medfro_LateRdm[0], beta_medfro_EarlyLateAligned[0])
                beta_medfro_laterdm_diff.append(diffevks)
                beta_medfro_laterdm_diff = beta_medfro_laterdm_diff[0]
                
                full_diffevks = np.subtract(full_beta_medfro_LateRdm[0], full_beta_medfro_EarlyLateAligned[0])
                full_beta_medfro_laterdm_diff.append(full_diffevks)
                full_beta_medfro_laterdm_diff = full_beta_medfro_laterdm_diff[0]
                
            elif cond == 4:
                diffevks = np.subtract(beta_medfro_EarlyMir[0], beta_medfro_EarlyLateAligned[0])
                beta_medfro_earlymir_diff.append(diffevks)
                beta_medfro_earlymir_diff = beta_medfro_earlymir_diff[0]
                
                full_diffevks = np.subtract(full_beta_medfro_EarlyMir[0], full_beta_medfro_EarlyLateAligned[0])
                full_beta_medfro_earlymir_diff.append(full_diffevks)
                full_beta_medfro_earlymir_diff = full_beta_medfro_earlymir_diff[0]
                
            elif cond == 5:
                diffevks = np.subtract(beta_medfro_LateMir[0], beta_medfro_EarlyLateAligned[0])
                beta_medfro_latemir_diff.append(diffevks)
                beta_medfro_latemir_diff = beta_medfro_latemir_diff[0]
                
                full_diffevks = np.subtract(full_beta_medfro_LateMir[0], full_beta_medfro_EarlyLateAligned[0])
                full_beta_medfro_latemir_diff.append(full_diffevks)
                full_beta_medfro_latemir_diff = full_beta_medfro_latemir_diff[0]

In [ ]:
# save aligned (baseline) subtracted tfr data across participants in each condition (used for plotting signals)
theta_flists = [full_theta_medfro_earlyrot_diff, full_theta_medfro_laterot_diff, 
                full_theta_medfro_earlyrdm_diff, full_theta_medfro_laterdm_diff, 
                full_theta_medfro_earlymir_diff, full_theta_medfro_latemir_diff]
alpha_flists = [full_alpha_medfro_earlyrot_diff, full_alpha_medfro_laterot_diff, 
                full_alpha_medfro_earlyrdm_diff, full_alpha_medfro_laterdm_diff, 
                full_alpha_medfro_earlymir_diff, full_alpha_medfro_latemir_diff]
beta_flists = [full_beta_medfro_earlyrot_diff, full_beta_medfro_laterot_diff, 
               full_beta_medfro_earlyrdm_diff, full_beta_medfro_laterdm_diff, 
               full_beta_medfro_earlymir_diff, full_beta_medfro_latemir_diff]

theta_conditionnames = ['theta_earlyrot', 'theta_laterot', 'theta_earlyrdm', 'theta_laterdm', 'theta_earlymir', 'theta_latemir']
alpha_conditionnames = ['alpha_earlyrot', 'alpha_laterot', 'alpha_earlyrdm', 'alpha_laterdm', 'alpha_earlymir', 'alpha_latemir']
beta_conditionnames = ['beta_earlyrot', 'beta_laterot', 'beta_earlyrdm', 'beta_laterdm', 'beta_earlymir', 'beta_latemir']

freq_bands = ['theta', 'alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(theta_flists)):
            df = pd.DataFrame(theta_flists[f]).T #no need to index [0]
            df['time'] = full_time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_%s_%s.csv' % (theta_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 1:
        for f in range(0, len(alpha_flists)):
            df = pd.DataFrame(alpha_flists[f]).T #no need to index [0]
            df['time'] = full_time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_%s_%s.csv' % (alpha_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 2:
        for f in range(0, len(beta_flists)):
            df = pd.DataFrame(beta_flists[f]).T
            df['time'] = full_time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_%s_%s.csv' % (beta_conditionnames[f], erps))
            df.to_csv(filename)

In [ ]:
# Compare EARLY VS LATE for each perturbation
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in plots
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

freq_bands = ['theta', 'alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        theta_conditionnames = ['theta_medfro_rot', 'theta_medfro_rdm', 'theta_medfro_mir']
        for c in range(0, len(theta_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(theta_medfro_laterot_diff, theta_medfro_earlyrot_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(theta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(theta_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(theta_medfro_laterdm_diff, theta_medfro_earlyrdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(theta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(theta_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(theta_medfro_latemir_diff, theta_medfro_earlymir_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(theta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(theta_conditionnames[c])
    elif band == 1:
        alpha_conditionnames = ['alpha_medfro_rot', 'alpha_medfro_rdm', 'alpha_medfro_mir']
        for c in range(0, len(alpha_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_laterot_diff, alpha_medfro_earlyrot_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_laterdm_diff, alpha_medfro_earlyrdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_latemir_diff, alpha_medfro_earlymir_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
    elif band == 2:
        beta_conditionnames = ['beta_medfro_rot', 'beta_medfro_rdm', 'beta_medfro_mir']
        for c in range(0, len(beta_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_laterot_diff, beta_medfro_earlyrot_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_laterdm_diff, beta_medfro_earlyrdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_latemir_diff, beta_medfro_earlymir_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_Permutation_test_EarlyvsLate_%s_%s.csv' % (erps, roi))
perm_test.to_csv(perm_test_filename)

In [ ]:
# Next step is to subtract early from late condition, to generate a single signal for each perturbation
diffconds = ['rot', 'rdm', 'mir']
freq_bands = ['theta', 'alpha', 'beta']

theta_medfro_rot_diff = []
theta_medfro_rdm_diff = []
theta_medfro_mir_diff = []
full_theta_medfro_rot_diff = []
full_theta_medfro_rdm_diff = []
full_theta_medfro_mir_diff = []

alpha_medfro_rot_diff = []
alpha_medfro_rdm_diff = []
alpha_medfro_mir_diff = []
full_alpha_medfro_rot_diff = []
full_alpha_medfro_rdm_diff = []
full_alpha_medfro_mir_diff = []

beta_medfro_rot_diff = []
beta_medfro_rdm_diff = []
beta_medfro_mir_diff = []
full_beta_medfro_rot_diff = []
full_beta_medfro_rdm_diff = []
full_beta_medfro_mir_diff = []

for band in range(0, len(freq_bands)):
    if band == 0:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(theta_medfro_laterot_diff, theta_medfro_earlyrot_diff)
                theta_medfro_rot_diff.append(diffevks)
                theta_medfro_rot_diff = theta_medfro_rot_diff[0] #to keep shape of object consistent
                
                full_diffevks = np.subtract(full_theta_medfro_laterot_diff, full_theta_medfro_earlyrot_diff)
                full_theta_medfro_rot_diff.append(full_diffevks)
                full_theta_medfro_rot_diff = full_theta_medfro_rot_diff[0] #to keep shape of object consistent
                
            elif cond == 1:
                diffevks = np.subtract(theta_medfro_laterdm_diff, theta_medfro_earlyrdm_diff)
                theta_medfro_rdm_diff.append(diffevks)
                theta_medfro_rdm_diff = theta_medfro_rdm_diff[0]
                
                full_diffevks = np.subtract(full_theta_medfro_laterdm_diff, full_theta_medfro_earlyrdm_diff)
                full_theta_medfro_rdm_diff.append(full_diffevks)
                full_theta_medfro_rdm_diff = full_theta_medfro_rdm_diff[0]
                
            elif cond == 2:
                diffevks = np.subtract(theta_medfro_latemir_diff, theta_medfro_earlymir_diff)
                theta_medfro_mir_diff.append(diffevks)
                theta_medfro_mir_diff = theta_medfro_mir_diff[0]
                
                full_diffevks = np.subtract(full_theta_medfro_latemir_diff, full_theta_medfro_earlymir_diff)
                full_theta_medfro_mir_diff.append(full_diffevks)
                full_theta_medfro_mir_diff = full_theta_medfro_mir_diff[0]
                
    elif band == 1:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(alpha_medfro_laterot_diff, alpha_medfro_earlyrot_diff)
                alpha_medfro_rot_diff.append(diffevks)
                alpha_medfro_rot_diff = alpha_medfro_rot_diff[0] #to keep shape of object consistent
                
                full_diffevks = np.subtract(full_alpha_medfro_laterot_diff, full_alpha_medfro_earlyrot_diff)
                full_alpha_medfro_rot_diff.append(full_diffevks)
                full_alpha_medfro_rot_diff = full_alpha_medfro_rot_diff[0] #to keep shape of object consistent
                
            elif cond == 1:
                diffevks = np.subtract(alpha_medfro_laterdm_diff, alpha_medfro_earlyrdm_diff)
                alpha_medfro_rdm_diff.append(diffevks)
                alpha_medfro_rdm_diff = alpha_medfro_rdm_diff[0]
                
                full_diffevks = np.subtract(full_alpha_medfro_laterdm_diff, full_alpha_medfro_earlyrdm_diff)
                full_alpha_medfro_rdm_diff.append(full_diffevks)
                full_alpha_medfro_rdm_diff = full_alpha_medfro_rdm_diff[0]
                
            elif cond == 2:
                diffevks = np.subtract(alpha_medfro_latemir_diff, alpha_medfro_earlymir_diff)
                alpha_medfro_mir_diff.append(diffevks)
                alpha_medfro_mir_diff = alpha_medfro_mir_diff[0]
                
                full_diffevks = np.subtract(full_alpha_medfro_latemir_diff, full_alpha_medfro_earlymir_diff)
                full_alpha_medfro_mir_diff.append(full_diffevks)
                full_alpha_medfro_mir_diff = full_alpha_medfro_mir_diff[0]
                
    elif band == 2:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(beta_medfro_laterot_diff, beta_medfro_earlyrot_diff)
                beta_medfro_rot_diff.append(diffevks)
                beta_medfro_rot_diff = beta_medfro_rot_diff[0] #to keep shape of object consistent
                
                full_diffevks = np.subtract(full_beta_medfro_laterot_diff, full_beta_medfro_earlyrot_diff)
                full_beta_medfro_rot_diff.append(full_diffevks)
                full_beta_medfro_rot_diff = full_beta_medfro_rot_diff[0] #to keep shape of object consistent
                
            elif cond == 1:
                diffevks = np.subtract(beta_medfro_laterdm_diff, beta_medfro_earlyrdm_diff)
                beta_medfro_rdm_diff.append(diffevks)
                beta_medfro_rdm_diff = beta_medfro_rdm_diff[0]
                
                full_diffevks = np.subtract(full_beta_medfro_laterdm_diff, full_beta_medfro_earlyrdm_diff)
                full_beta_medfro_rdm_diff.append(full_diffevks)
                full_beta_medfro_rdm_diff = full_beta_medfro_rdm_diff[0]
                
            elif cond == 2:
                diffevks = np.subtract(beta_medfro_latemir_diff, beta_medfro_earlymir_diff)
                beta_medfro_mir_diff.append(diffevks)
                beta_medfro_mir_diff = beta_medfro_mir_diff[0]
                
                full_diffevks = np.subtract(full_beta_medfro_latemir_diff, full_beta_medfro_earlymir_diff)
                full_beta_medfro_mir_diff.append(full_diffevks)
                full_beta_medfro_mir_diff = full_beta_medfro_mir_diff[0]

In [ ]:
# save early and late subtracted tfr data across participants in each condition (used for plotting signals)
theta_flists = [full_theta_medfro_rot_diff, full_theta_medfro_rdm_diff, full_theta_medfro_mir_diff]
alpha_flists = [full_alpha_medfro_rot_diff, full_alpha_medfro_rdm_diff, full_alpha_medfro_mir_diff]
beta_flists = [full_beta_medfro_rot_diff, full_beta_medfro_rdm_diff, full_beta_medfro_mir_diff]

theta_conditionnames = ['theta_rot', 'theta_rdm', 'theta_mir']
alpha_conditionnames = ['alpha_rot', 'alpha_rdm', 'alpha_mir']
beta_conditionnames = ['beta_rot', 'beta_rdm', 'beta_mir']

freq_bands = ['theta', 'alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(theta_flists)):
            df = pd.DataFrame(theta_flists[f]).T #no need to index [0]
            df['time'] = full_time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_EvL_%s_%s.csv' % (theta_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 1:
        for f in range(0, len(alpha_flists)):
            df = pd.DataFrame(alpha_flists[f]).T #no need to index [0]
            df['time'] = full_time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_EvL_%s_%s.csv' % (alpha_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 2:
        for f in range(0, len(beta_flists)):
            df = pd.DataFrame(beta_flists[f]).T
            df['time'] = full_time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_EvL_%s_%s.csv' % (beta_conditionnames[f], erps))
            df.to_csv(filename)

In [ ]:
# Compare EARLY VS LATE for each perturbation
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in plots
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

freq_bands = ['theta', 'alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        theta_conditionnames = ['theta_medfro_rotvmir', 'theta_medfro_rotvrdm', 'theta_medfro_mirvrdm']
        for c in range(0, len(theta_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(theta_medfro_rot_diff, theta_medfro_mir_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(theta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(theta_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(theta_medfro_rot_diff, theta_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(theta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(theta_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(theta_medfro_mir_diff, theta_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(theta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(theta_conditionnames[c])
    elif band == 1:
        alpha_conditionnames = ['alpha_medfro_rotvmir', 'alpha_medfro_rotvrdm', 'alpha_medfro_mirvrdm']
        for c in range(0, len(alpha_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_rot_diff, alpha_medfro_mir_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_rot_diff, alpha_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_mir_diff, alpha_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
    elif band == 2:
        beta_conditionnames = ['beta_medfro_rotvmir', 'beta_medfro_rotvrdm', 'beta_medfro_mirvrdm']
        for c in range(0, len(beta_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_rot_diff, beta_medfro_mir_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_rot_diff, beta_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_mir_diff, beta_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_Permutation_test_PerturbTypeComp_%s_%s.csv' % (erps, roi))
perm_test.to_csv(perm_test_filename)